In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from pycocotools.coco import COCO
import torchvision.transforms.functional as TF

from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from effdet.efficientdet import HeadNet
from timm.utils import AverageMeter


In [ ]:
class COCODetectionPaddedDataset(Dataset):
    def __init__(self, images_dir, annotation_path):
        self.images_dir = images_dir
        self.coco = COCO(annotation_path)
        self.image_ids = list(self.coco.imgs.keys())

    def __len__(self):
        return len(self.image_ids)

    def pad_to_square(self, image, target):
        w, h = image.size  # original size (1024, 128)
        max_dim = max(w, h)  # 1024

        # Compute padding (left, top, right, bottom)
        pad_left = 0
        pad_top = (max_dim - h) // 2
        pad_right = 0
        pad_bottom = max_dim - h - pad_top

        # Apply padding to image
        image = TF.pad(image, (pad_left, pad_top, pad_right, pad_bottom), fill=0)

        # Apply same to boxes
        boxes = target['boxes']
        boxes[:, 1] += pad_top
        boxes[:, 3] += pad_top

        return image, target

    def __getitem__(self, index):
        coco = self.coco
        image_id = self.image_ids[index]
        ann_ids = coco.getAnnIds(imgIds=image_id)
        anns = coco.loadAnns(ann_ids)

        # Load image
        img_info = coco.loadImgs(image_id)[0]
        img_path = os.path.join(self.images_dir, img_info['file_name'])
        image = Image.open(img_path).convert("RGB")

        # Load boxes and labels
        boxes = []
        labels = []
        for ann in anns:
            if 'bbox' in ann:
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x + w, y + h])
                labels.append(0)  # Only one class → label = 0

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([image_id])
        }

        # Pad image + boxes
        image, target = self.pad_to_square(image, target)

        # Normalize
        image = TF.to_tensor(image)
        image = TF.normalize(image, mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])

        return image, target


In [ ]:
from torch.utils.data import DataLoader


train_dataset = COCODetectionPaddedDataset(
    images_dir='./lidar_dataset_links/train/images',
    annotation_path='lidar_train_coco.json'
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


val_dataset = COCODetectionPaddedDataset(
    images_dir='./lidar_dataset_links/valid/images',
    annotation_path='lidar_valid_coco.json'
)

val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import torch
from torch.utils.data import DataLoader

# --- Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 1
epochs = 5
batch_size = 5
img_size = (1024, 1024)

# --- Model creation ---
config = get_efficientdet_config('tf_efficientdet_d2')
config.num_classes = num_classes
config.image_size = img_size
config.norm_kwargs = dict(eps=0.001, momentum=0.01)

net = EfficientDet(config, pretrained_backbone=True)
net.class_net = HeadNet(config, num_outputs=num_classes)
model = DetBenchTrain(net, config)
model.to(device)

# --- Optimizer ---
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# --- Training loop ---
for epoch in range(epochs):
    model.train()
    loss_meter = AverageMeter()

    for images, targets in train_loader:  # From your custom dataset
        images = torch.stack([img.to(device) for img in images])

        # Move target boxes and labels to device
        boxes = [t['boxes'].to(device) for t in targets]
        labels = [t['labels'].to(device) for t in targets]
        targets_dict = {'bbox': boxes, 'cls': labels}

        # EfficientDet expects list of dicts with 'boxes' and 'labels'
        loss_dict = model(images, targets_dict)
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_meter.update(loss.item(), len(images))

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss_meter.avg:.4f}")


In [ ]:
torch.save(model, "efficientdet_model.pth")

In [ ]:
model = torch.load("efficientdet_model.pth",weights_only=False)

In [ ]:
from tqdm import tqdm

def run_inference(model, dataloader):
    model.eval()
    results = []

    for images, targets in tqdm(dataloader):
        images = torch.stack([img.to('cuda') for img in images])
        image_ids = [t['image_id'].item() for t in targets]

        with torch.no_grad():
            outputs = model(images)

        for i, output in enumerate(outputs):
            output = output.cpu()
            boxes = output[:, :4]
            scores = output[:, 4]
            labels = output[:, 5].int()

            results.append({
                'image_id': image_ids[i],
                'boxes': boxes,
                'scores': scores,
                'labels': labels
            })

    return results


from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json


def convert_to_coco(results, score_threshold=0.05):
    coco_results = []
    pad_top = 448  

    for res in results:
        for box, score, label in zip(res['boxes'], res['scores'], res['labels']):
            x1, y1, x2, y2 = box.tolist()

            y1 -= pad_top
            y2 -= pad_top

            y1 = max(0, y1)
            y2 = min(128, y2)

            width = x2 - x1
            height = y2 - y1

            if score >= score_threshold and width > 1 and height > 1:
                coco_results.append({
                    'image_id': int(res['image_id']),
                    'category_id': int(label),  
                    'bbox': [x1, y1, width, height],
                    'score': float(score)
                })
    return coco_results


def evaluate_coco(gt_annotations_file, predictions):
    json_filename = 'predictionsNOOB.json'
    with open(json_filename, 'w') as f:
        json.dump(predictions, f)

    coco_gt = COCO(gt_annotations_file)
    coco_dt = coco_gt.loadRes(json_filename)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()


In [ ]:
from effdet import DetBenchPredict


eval_model = DetBenchPredict(model.model).to(device)
eval_model.eval()
val_results = run_inference(eval_model, val_loader)
coco_results = convert_to_coco(val_results,0.1)
evaluate_coco('valid_coco.json', coco_results)

In [ ]:
coco_gt = COCO('valid_coco copy.json')
coco_dt = coco_gt.loadRes('predictionsNOOB.json')
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()